In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM mapping_categories WHERE categorie IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
PCLAT,6219,PERCHLORATE,Perchlorate,14797-73-0,perchlorate


In [3]:
%%sql
SELECT * FROM mapping_categories
WHERE libminparametre IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
14DAN,1580,"1,4 DIOXANE","1,4 dioxane",123-91-1,hydrocarbures


In [4]:
%%sql
SELECT DISTINCT 
    cdparametresiseeaux, limitequal_float , limitequal
FROM int__resultats_udi_communes
WHERE cdparametresiseeaux IN ('14DAN','PCLAT')

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdparametresiseeaux,limitequal_float,limitequal
PCLAT,None,None
14DAN,None,None


### Dernier relevé

In [5]:
%%sql --save int__resultats_sub_indus_udi_dernier

WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux IN ('14DAN', 'PCLAT')
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
)

SELECT
    last_pvl.cdreseau,
    'dernier_prel' AS periode,
    last_pvl.datetimeprel AS dernier_prel_datetime,
    last_pvl.valtraduite AS dernier_prel_valeur,
    1 AS nb_parametres,
    'sub_indus_' || last_pvl.cdparametresiseeaux AS categorie,
    CASE
        WHEN
            last_pvl.valtraduite = 0
            OR last_pvl.valtraduite IS NULL
            THEN 'non_quantifie'
        WHEN
            last_pvl.valtraduite >= 0.35
            AND last_pvl.cdparametresiseeaux = '14DAN'
            THEN 'sup_0_35'
        WHEN
            last_pvl.valtraduite < 0.35
            AND last_pvl.cdparametresiseeaux = '14DAN'
            THEN 'inf_0_35'
        WHEN
            last_pvl.valtraduite >= 15
            AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'sup_15'
        WHEN
            last_pvl.valtraduite >= 4 AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'sup_4'
        WHEN
            last_pvl.valtraduite < 4 AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'inf_4'
        WHEN
            last_pvl.valtraduite < 15 AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'inf_15'
        ELSE 'error'
    END AS resultat
FROM
    last_pvl
WHERE
    last_pvl.row_number = 1



Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,periode,dernier_prel_datetime,dernier_prel_valeur,nb_parametres,categorie,resultat
002000045,dernier_prel,2024-10-22 09:58:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000136,dernier_prel,2024-10-23 12:13:00,1.700,1,sub_indus_PCLAT,inf_4
002000155,dernier_prel,2024-10-07 10:46:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000318,dernier_prel,2024-04-15 09:39:00,6.100,1,sub_indus_PCLAT,sup_4
002000463,dernier_prel,2024-07-02 11:24:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000583,dernier_prel,2024-04-22 09:02:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000716,dernier_prel,2024-05-29 09:16:00,2.900,1,sub_indus_PCLAT,inf_4
002000824,dernier_prel,2024-07-01 12:25:00,2.800,1,sub_indus_PCLAT,inf_4
002000925,dernier_prel,2024-11-20 10:00:00,6.100,1,sub_indus_PCLAT,sup_4
002000963,dernier_prel,2024-07-17 10:03:00,0.000,1,sub_indus_PCLAT,non_quantifie


In [6]:
%%sql --with int__resultats_sub_indus_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_sub_indus_udi_dernier
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,periode,dernier_prel_datetime,dernier_prel_valeur,nb_parametres,categorie,resultat


In [7]:
%%sql --with int__resultats_sub_indus_udi_dernier
SELECT 
    categorie, resultat, COUNT(*) 
FROM 
    int__resultats_sub_indus_udi_dernier
GROUP BY 
    categorie, resultat

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

categorie,resultat,count_star()
sub_indus_PCLAT,sup_4,182
sub_indus_PCLAT,sup_15,29
sub_indus_PCLAT,inf_4,429
sub_indus_PCLAT,non_quantifie,193
sub_indus_14DAN,sup_0_35,1
sub_indus_14DAN,non_quantifie,1


In [8]:
%%sql --with int__resultats_sub_indus_udi_dernier
SELECT 
    categorie, resultat, MIN( cdreseau) min_cdreseau, MAX(cdreseau) AS max_cdreseau
FROM 
    int__resultats_sub_indus_udi_dernier
GROUP BY 
    categorie, resultat

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

categorie,resultat,min_cdreseau,max_cdreseau
sub_indus_PCLAT,sup_15,002000114,080003918
sub_indus_PCLAT,sup_4,002000060,088002246
sub_indus_PCLAT,inf_4,002000048,080003913
sub_indus_PCLAT,non_quantifie,002000045,080001512
sub_indus_14DAN,non_quantifie,076000543,076000543
sub_indus_14DAN,sup_0_35,052000760,052000760


In [9]:
%%sql --with int__resultats_sub_indus_udi_dernier 
 SELECT 
   *
 FROM
   int__resultats_sub_indus_udi_dernier
WHERE
    cdreseau IN( '002000114','002000060','002000048','002000045','052000760','076000543',
    '080003918','088002246','080003913','080001512','052000760','076000543')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,periode,dernier_prel_datetime,dernier_prel_valeur,nb_parametres,categorie,resultat
002000045,dernier_prel,2024-10-22 09:58:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000048,dernier_prel,2024-04-22 10:10:00,1.900,1,sub_indus_PCLAT,inf_4
002000060,dernier_prel,2024-10-18 08:31:00,8.300,1,sub_indus_PCLAT,sup_4
002000114,dernier_prel,2024-10-10 11:03:00,45.000,1,sub_indus_PCLAT,sup_15
052000760,dernier_prel,2024-12-13 14:52:00,0.840,1,sub_indus_14DAN,sup_0_35
076000543,dernier_prel,2024-11-05 10:15:00,0.000,1,sub_indus_14DAN,non_quantifie
080001512,dernier_prel,2024-05-02 14:30:00,0.000,1,sub_indus_PCLAT,non_quantifie
080003913,dernier_prel,2024-05-31 11:23:00,2.200,1,sub_indus_PCLAT,inf_4
080003918,dernier_prel,2024-10-23 13:09:00,29.000,1,sub_indus_PCLAT,sup_15
088002246,dernier_prel,2024-10-22 10:33:00,9.200,1,sub_indus_PCLAT,sup_4


### Annuel

In [10]:
%%sql --save int__resultats_sub_indus_udi_annuel
WITH
prels AS (
    -- Certains prélèvements ont plusieurs analyses pour la même substance
    -- Le SELECT DISTINCT ne dédoublonne pas ces cas là
    -- Donc il n'y a pas d'unicité sur referenceprel dans cetre requête
    SELECT DISTINCT
        de_partition AS annee,
        cdreseau,
        cdparametresiseeaux,
        referenceprel,
        datetimeprel,
        valtraduite
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux IN ('14DAN', 'PCLAT')
)

SELECT
    cdreseau,
    annee,
    'sub_indus_' || cdparametresiseeaux AS categorie,
    'bilan_annuel_' || annee AS periode,
    count(
        DISTINCT
        CASE
            WHEN
                (
                    valtraduite IS NOT NULL
                    AND valtraduite >= 0.35
                    AND cdparametresiseeaux = '14DAN'
                )
                OR (
                    valtraduite IS NOT NULL
                    AND valtraduite >= 4
                    AND cdparametresiseeaux = 'PCLAT'
                )
                THEN referenceprel
        END
    ) AS nb_depassements,
    count(DISTINCT referenceprel) AS nb_prelevements,
    (
        count(
            DISTINCT
            CASE
                WHEN
                    (
                        valtraduite IS NOT NULL
                        AND valtraduite >= 0.35
                        AND cdparametresiseeaux = '14DAN'
                    )
                    OR (
                        valtraduite IS NOT NULL
                        AND valtraduite >= 4
                        AND cdparametresiseeaux = 'PCLAT'
                    )
                    THEN referenceprel
            END
        )::float
        /
        count(DISTINCT referenceprel)::float
    ) AS ratio_depassements

FROM prels

GROUP BY cdreseau, annee, categorie


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_depassements
002001048,2022,sub_indus_PCLAT,bilan_annuel_2022,0,1,0.0
027000441,2020,sub_indus_PCLAT,bilan_annuel_2020,0,14,0.0
002001353,2021,sub_indus_PCLAT,bilan_annuel_2021,0,3,0.0
060000626,2022,sub_indus_PCLAT,bilan_annuel_2022,0,1,0.0
080000577,2022,sub_indus_PCLAT,bilan_annuel_2022,1,1,1.0
002000839,2023,sub_indus_PCLAT,bilan_annuel_2023,1,1,1.0
062000640,2023,sub_indus_PCLAT,bilan_annuel_2023,0,1,0.0
062000546,2023,sub_indus_PCLAT,bilan_annuel_2023,0,1,0.0
060001451,2024,sub_indus_PCLAT,bilan_annuel_2024,0,1,0.0
062000750,2024,sub_indus_PCLAT,bilan_annuel_2024,0,2,0.0


In [11]:
%%sql --with int__resultats_sub_indus_udi_annuel 
 SELECT 
   *
 FROM
   int__resultats_sub_indus_udi_annuel
WHERE
    cdreseau IN( '002000114','002000060','002000048','002000045','052000760','076000543',
    '080003918','088002246','080003913','080001512','052000760','076000543')
    AND nb_prelevements > 1
    AND nb_depassements > 0
ORDER BY 
    ratio_depassements, cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_depassements
076000543,2024,sub_indus_14DAN,bilan_annuel_2024,1,9,0.1111111119389534
076000543,2023,sub_indus_14DAN,bilan_annuel_2023,5,12,0.4166666567325592
088002246,2022,sub_indus_PCLAT,bilan_annuel_2022,1,2,0.5
076000543,2022,sub_indus_14DAN,bilan_annuel_2022,10,12,0.8333333134651184
080003918,2023,sub_indus_PCLAT,bilan_annuel_2023,2,2,1.0
080003918,2022,sub_indus_PCLAT,bilan_annuel_2022,2,2,1.0
080003918,2024,sub_indus_PCLAT,bilan_annuel_2024,2,2,1.0
080003918,2020,sub_indus_PCLAT,bilan_annuel_2020,2,2,1.0
080003918,2021,sub_indus_PCLAT,bilan_annuel_2021,2,2,1.0
088002246,2024,sub_indus_PCLAT,bilan_annuel_2024,2,2,1.0


### Tests

In [12]:
%%sql 
-- dernier relevé
SELECT
*
FROM
    int__resultats_sub_indus_udi_dernier
WHERE
    (
        cdreseau = '002000045'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-10-22 09:58:00'
        AND resultat != 'non_quantifie'
    )
    OR
    (
        cdreseau = '002000048'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-04-22 10:10:00'
        AND resultat != 'inf_4'
    )
    OR
    (
        cdreseau = '002000060'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-10-18 08:31:00'
        AND resultat != 'sup_4'
    )
    OR
    (
        cdreseau = '002000114'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-10-10 11:03:00'
        AND resultat != 'sup_15'
    )
    OR
    (
        cdreseau = '052000760'
        AND categorie = 'sub_indus_14DAN'
        AND dernier_prel_datetime = '2024-12-13 14:52:00'
        AND resultat != 'sup_0_35'
    )
    OR
    (
        cdreseau = '076000543'
        AND categorie = 'sub_indus_14DAN'
        AND dernier_prel_datetime = '2024-11-05 10:15:00'
        AND resultat != 'non_quantifie'
    )
    OR
    (
        cdreseau = '080001512'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-05-02 14:30:00'
        AND resultat != 'non_quantifie'
    )
    OR
    (
        cdreseau = '080003913'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-05-31 11:23:00'
        AND resultat != 'inf_4'
    )
    OR
    (
        cdreseau = '080003918'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-10-23 13:09:00'
        AND resultat != 'sup_15'
    )
    OR
    (
        cdreseau = '088002246'
        AND categorie = 'sub_indus_PCLAT'
        AND dernier_prel_datetime = '2024-10-22 10:33:00'
        AND resultat != 'sup_4'
    )



Generating CTE with stored snippets: 'int__resultats_sub_indus_udi_dernier'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

periode,cdreseau,categorie,resultat,nb_depassements,nb_prelevements,ratio_depassements


In [15]:
%%sql 
-- annuel
SELECT
 *
FROM
    int__resultats_sub_indus_udi_annuel
WHERE
    (
        cdreseau = '068006326'
        AND categorie = 'sub_indus_PCLAT'
        AND annee = '2020'
        AND nb_depassements != 0
    )
    OR
    (
        cdreseau = '002001026'
        AND categorie = 'sub_indus_PCLAT'
        AND annee = '2021'
        AND nb_depassements != 0
    )
    OR
    (
        cdreseau = '062000680'
        AND categorie = 'sub_indus_PCLAT'
        AND annee = '2022'
        AND nb_depassements != 0
    )
    OR
    (
        cdreseau = '028001088'
        AND categorie = 'sub_indus_PCLAT'
        AND annee = '2023'
        AND (
            nb_prelevements != 28
            OR nb_depassements != 11
        )
    )
    OR
    (
        cdreseau = '052000760'
        AND categorie = 'sub_indus_14DAN'
        AND annee = '2024'
        AND (
            nb_prelevements != 1
            OR nb_depassements != 1
        )
    )
    OR
    (
        cdreseau = '076000543'
        AND categorie = 'sub_indus_14DAN'
        AND annee IN ('2022', '2023')
        AND nb_prelevements != 12
    )
    OR
    (
        cdreseau = '076000543'
        AND categorie = 'sub_indus_14DAN'
        AND annee = '2024'
        AND nb_prelevements != 9
    )
    OR
    (
        cdreseau = '088002246'
        AND categorie = 'sub_indus_PCLAT'
        AND annee = '2022'
        AND (
            ratio_depassements != 0.5
            OR
            nb_prelevements != 2
        )
    )

Generating CTE with stored snippets: 'int__resultats_sub_indus_udi_annuel'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_depassements
